## Enviroment Setup

Nesta primeira célula, estamos reunindo as “peças” essenciais que serão os alicerces do nosso notebook de padronização de dados de produtos. Pense nela como a bancada de laboratório onde você prepara todos os reagentes antes de começar o experimento:

import json

Carrega e salva configurações e resultados em formato JSON, nossa linguagem universal para trocar informações entre sistemas.

LangChain & Chroma (Vector Store)

from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

Aqui trazemos o Chroma, que vai armazenar vetores de embedding dos textos dos produtos, e o SemanticSimilarityExampleSelector, nosso “curador” que escolhe exemplos mais relevantes com base em similaridade semântica.

Templates de Prompt

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.few_shot import FewShotChatMessagePromptTemplate

Essas classes permitem montar conversas estruturadas (prompts) com o modelo de IA. É nelas que definiremos o “roteiro” criativo para o ChatGPT transformar descrições livres em campos padronizados.

OpenAI & Embeddings

from langchain_openai import ChatOpenAI, OpenAIEmbeddings

Aqui é onde chamamos a mágica do ChatOpenAI para gerar texto e do OpenAIEmbeddings para converter frases dos produtos em vetores numéricos que o Chroma entende.

Dotenv

from dotenv import load_dotenv

Garante que suas chaves de API e outras variáveis sensíveis fiquem guardadas num arquivo .env, sem vazar no código.

Modelo de Domínio

from product_model import Product

Importa a classe Product, que define como queremos que nossos dados padronizados de produto sejam estruturados: nome, categoria, atributos, etc.

Pandas & NumPy

import pandas as pd
import numpy as np

São nossas lentes e escalpelo para ler, fatiar, transformar e manipular tabelas e matrizes de dados com agilidade.

🌟 Por que isso importa?
Ao juntar essas bibliotecas, temos:

Leitura/Gravação de configurações (JSON).

Armazenamento de vetores semânticos (Chroma).

Seleção Inteligente de exemplos (SemanticSimilarity).

Montagem de prompts dinâmicos (ChatPromptTemplate).

Chamada ao ponto de entrada da IA (ChatOpenAI).

Proteção das credenciais (dotenv).

Estruturação do resultado (Product).

Manipulação de dados em massa (pandas & numpy).

In [21]:
import json

from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from dotenv import load_dotenv
from product_model import Product

import pandas as pd
import numpy as np

load_dotenv(): carrega variáveis de ambiente a partir do arquivo .env.

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini"): inicializa o GPT-4o-mini com temperatura zero, garantindo saídas consistentes e reproduzíveis.

In [22]:
load_dotenv()
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

## Load Examples

Propósito: transforma cada linha do DataFrame num par input/output pronto para alimentar o pipeline de IA.

“input”: pega o texto livre em row["description"].

“output”: gera uma string JSON (com acentuação preservada) contendo os campos padronizados do Product — tipo, marcas, variação, embalagem, unidade e quantidade.

Em resumo, esta função empacota seu dado bruto e a estrutura esperada num exemplo único, ideal para treinar ou chamar o modelo de forma consistente.

In [23]:
def row_to_json(row):
    return {
        "input": row["description"],
        "output": json.dumps(
            {
                "type": row.get("type"),
                "primary_brand": row.get("primary_brand"),
                "secondary_brand": row.get("secondary_brand"),
                "variation": row.get("variation"),
                "container": row.get("container"),
                "container_type": row.get("container_type"),
                "measure": row.get("measure"),
                "unity": row.get("unity"),
                "amount": row.get("amount"),
            },
            ensure_ascii=False,
        ),
    }

Lê o arquivo examples.csv em um DataFrame.

Converte valores NaN em None para compatibilidade com JSON e IA.

Exibe as primeiras linhas (head) para verificação rápida dos exemplos disponíveis.

In [24]:
examples_df = pd.read_csv("examples.csv").replace({np.nan: None})
examples_df.head()

,description,type,primary_brand,secondary_brand,variation,container,container_type,amount,measure,unity
0,CERVEJA ANTARCTICA PILSEN GARRAFA VIDRO 600ML,cerveja,antarctica,None,pilsen,garrafa,vidro,1,600.0,ml
1,CERVEJA PATAGONIA BOHEMIAN PILSENER GARRAFA ON...,cerveja,patagonia,None,bohemian pilsener,garrafa,one way,1,740.0,ml
2,CERVEJA SKOL BEATS SECRET LT 269ML,cerveja,skol,beats,secret,lata,None,1,269.0,ml
3,CERV. CARACU LN 355ML C/6,cerveja,caracu,None,None,long neck,None,6,355.0,ml
4,SUCO DE UVA ALIANÇA TINTO INTEGRAL 1.5L,suco,aliança,None,uva tinto integral,None,None,1,1.5,L


Usa row_to_json para mapear cada linha de examples_df a um dicionário {input, output}.

Converte a série resultante em uma lista Python (examples_json), pronta para alimentar prompts Few-Shot ou seletores semânticos.

In [25]:
examples_json = examples_df.apply(row_to_json, axis=1).tolist()

Cria um seletor que armazena os exemplos como vetores no Chroma, usando embeddings do modelo text-embedding-3-small.

k=2 garante que, a cada chamada, você receba os 2 exemplos mais semanticamente relevantes para enriquecer seu prompt ou few-shot.

In [26]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples_json,
    embeddings=OpenAIEmbeddings(model="text-embedding-3-small"),
    vectorstore_cls=Chroma,
    k=2,
)

## Create prompt

Cria um ChatPromptTemplate com duas mensagens encadeadas:

Humano recebe {input} (descrição bruta).

IA responde com {output} (JSON padronizado).

Esses placeholders serão substituídos dinamicamente pelos exemplos ou inputs reais no momento da chamada ao modelo.

In [27]:
example_prompt = ChatPromptTemplate.from_messages(
    [("human", "{input}"), ("ai", "{output}")]
)

Instancia um FewShotChatMessagePromptTemplate que combina:

example_selector: busca os 2 exemplos mais relevantes via similaridade semântica.

example_prompt: formata cada exemplo como pares “humano → IA” (descrição bruta e JSON padronizado).

Por que isso importa?
Ao chamar esse prompt, o sistema:

Seleciona automaticamente os exemplos mais próximos do seu item atual.

Insere esses pares formatados no início da conversa.

Garante que o modelo receba contexto personalizado, melhorando a precisão da padronização.

In [28]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
)

Sistema: define o papel do modelo (“expert extraction algorithm”) e força respostas em português.

Few-Shot: injeta automaticamente os 2 exemplos semânticos mais relevantes para guiar a extração.

Humano: insere a descrição real do produto ({input}) para que o modelo extraia e retorne o JSON padronizado.

Esse prompt une contexto, exemplos e entrada dinâmica, garantindo que o LLM entenda exatamente o papel e o formato de saída desejado.

In [29]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Your job is to retrieve information from the product description. "
            "Answer in portuguese and in portuguese only. "
            "Here are some examples on how to do this job:\n",
        ),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

## Calling chain

O que faz: envia ao llm o prompt completo (contexto de sistema, exemplos few-shot e o nosso novo input) usando prompt.invoke, e captura o fluxo de mensagens que o modelo gera.

Input de exemplo: "DESODORANTE REXONA MAN 120ml" um descritivo típico de produto.

Output: uma lista de objetos messages contendo:

A mensagem de sistema com instruções.

Os exemplos selecionados e formatados.

A mensagem do usuário com o input real.

A resposta da IA, que virá como JSON padronizado com campos como type, primary_brand, measure, etc.

Esse passo valida toda a configuração: garante que, a partir de uma descrição livre, o modelo retorne exatamente o JSON estruturado que você precisa para sua base de dados de produtos.

In [30]:
prompt.invoke({"input": "DESODORANTE REXONA MAN 120ml"}).messages

[SystemMessage(content='You are an expert extraction algorithm. Your job is to retrieve information from the product description. Answer in portuguese and in portuguese only. Here are some examples on how to do this job:\n', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='DESODORANTE REXONA V8 150ML', additional_kwargs={}, response_metadata={}),
 AIMessage(content='{"type": "desodorante", "primary_brand": "rexona", "secondary_brand": null, "variation": "v8", "container": null, "container_type": null, "measure": 150.0, "unity": "ml", "amount": 1}', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='DESODORANTE REXONA V8 150ML', additional_kwargs={}, response_metadata={}),
 AIMessage(content='{"type": "desodorante", "primary_brand": "rexona", "secondary_brand": null, "variation": "v8", "container": null, "container_type": null, "measure": 150.0, "unity": "ml", "amount": 1}', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='DESODORAN

O que faz: usa o operador | para encadear o prompt e o LLM configurado para saída estruturada.

llm.with_structured_output(schema=Product): instrui o modelo a devolver diretamente um objeto do tipo Product, respeitando o schema (campos e tipos) definidos na classe.

Vantagem: elimina parsing manual—você recebe instâncias de Product validadas e prontas para uso no seu fluxo de padronização.

In [31]:
chain = prompt | llm.with_structured_output(schema=Product)

response = chain.invoke({"input": "DESODORANTE REXONA MAN 120ml"}): executa todo o fluxo (prompt + LLM com saída estruturada) e retorna um objeto Product preenchido com os campos extraídos.

O response já é uma instância válida de Product, pronta para uso imediato em análises, gravação em banco ou exportação.

In [32]:
response = chain.invoke({"input": "DESODORANTE REXONA MAN 120ml"})

response.__dict__: acessa o dicionário interno do objeto Product, exibindo todos os atributos (tipo, marca, variação, medida, etc.) e seus valores extraídos.

Objetivo: validar de forma rápida e legível o conteúdo da instância retornada pela pipeline, conferindo que cada campo foi preenchido conforme esperado.

Uso prático: ideal para debugging ou antes de persistir os dados em um banco ou exportar para outro formato.

In [33]:
response.__dict__

{'type': 'desodorante',
 'primary_brand': 'rexona',
 'secondary_brand': None,
 'variation': 'man',
 'container': None,
 'container_type': None,
 'amount': 1,
 'measure': 120.0,
 'unity': 'ml'}